In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

#load dataset
file_path = "original_preprocess.csv"
df = pd.read_csv(file_path)

df.drop(columns=["date", "failed_experiment", "cpu_platform", "UL/DL", "rapl_var", "n_pm", "pm_median", "txgain_ul", "clockspeed", "cqi_dl", "bsr_ul", "num_ues", "bler_dl"], inplace=True, errors='ignore')

features_to_analyze = ["txgain_ul", "selected_mcs_ul", "selected_mcs_dl", "airtime_ul", "airtime_dl"]
target_variable = "pm_power"

results = {
    feature: {"value": [], "mae": [], "mse": [], "rmse": [], "mape": [],
              "relative_errors": [], "mean_actual": [], "mean_predicted": [],
              "std_relative_errors": [], "percentiles": {}}
    for feature in features_to_analyze
}

#loop through each feature
for feature in features_to_analyze:
    print(f"Processing feature: {feature}")
    unique_values = sorted(df[feature].unique())

    for value in unique_values:
        print(f"Processing value: {value}")

        df_value = df[df[feature] == value]

        if len(df_value) < 50:
            print(f"Skipping value {value} due to not enough data.")
            continue

        #separate features and target
        X = df_value.drop(columns=[target_variable] + features_to_analyze)
        y = df_value[target_variable]
        X = X.select_dtypes(include=[np.number])
        X_std = X.std().replace(0, 1)
        X = (X - X.mean()) / X_std

        #80-20 train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

        #Baseline DNN model
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(X_train.shape[1],)),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(1)
        ])

        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
        model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=16, verbose=0)

        #predict
        y_pred = model.predict(X_test).flatten()
        if np.isnan(y_pred).any():
            continue

        #calculate metrics
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

        #relative error for each test point
        relative_errors = np.abs((y_test - y_pred) / y_test) * 100
        mean_actual = np.mean(y_test)
        mean_predicted = np.mean(y_pred)
        std_relative_errors = np.std(relative_errors)

        #store results
        results[feature]["value"].append(value)
        results[feature]["mae"].append(mae)
        results[feature]["mse"].append(mse)
        results[feature]["rmse"].append(rmse)
        results[feature]["mape"].append(mape)
        results[feature]["relative_errors"].append(np.mean(relative_errors))
        results[feature]["std_relative_errors"].append(std_relative_errors)
        results[feature]["mean_actual"].append(mean_actual)
        results[feature]["mean_predicted"].append(mean_predicted)
        results[feature]["percentiles"][value] = {
            "25th Percentile": np.percentile(relative_errors, 25),
            "50th Percentile (Median)": np.percentile(relative_errors, 50),
            "75th Percentile": np.percentile(relative_errors, 75),
        }

#mean Relative Error % with standard deviation error bars
for feature in features_to_analyze:
    plt.figure(figsize=(12, 6))

    values = results[feature]["value"]
    mean_relative_errors = results[feature]["relative_errors"]
    std_relative_errors = results[feature]["std_relative_errors"]
    plt.errorbar(values, mean_relative_errors, yerr=std_relative_errors, fmt='o-', capsize=5, label='Relative Error (%)', color='blue')
    plt.xlabel("Feature value")
    plt.ylabel("Mean Relative Error (%)")
    plt.legend()
    plt.grid()
    plt.show()

#mean Actual against mean predicted power with std error bars
for feature in features_to_analyze:
    plt.figure(figsize=(12, 6))
    values = results[feature]["value"]
    mean_actual = results[feature]["mean_actual"]
    mean_predicted = results[feature]["mean_predicted"]
    std_predicted = results[feature]["std_relative_errors"]
    plt.errorbar(values, mean_actual, yerr=std_predicted, fmt='o-', capsize=5, label='Mean Actual Power (W)', color='red')
    plt.errorbar(values, mean_predicted, yerr=std_predicted, fmt='s-', capsize=5, label='Mean Predicted Power (W)', color='green')
    plt.xlabel("Feature value")
    plt.ylabel("Power (W)")
    plt.legend()
    plt.grid()
    plt.show()

#percentile values of relative errors
for feature in features_to_analyze:
    plt.figure(figsize=(12, 6))

    values = results[feature]["value"]
    percentile_data = [results[feature]["percentiles"][value] for value in values]

    #percentiles
    p25 = [p["25th Percentile"] for p in percentile_data]
    p50 = [p["50th Percentile (Median)"] for p in percentile_data]
    p75 = [p["75th Percentile"] for p in percentile_data]
    plt.plot(values, p25, 'b--', label='25th Percentile')
    plt.plot(values, p50, 'g-', label='50th Percentile (Median)')
    plt.plot(values, p75, 'r--', label='75th Percentile')
    plt.xlabel("Feature value")
    plt.ylabel("Mean Relative Error (%)")
    plt.legend()
    plt.grid()
    plt.show()

for feature in features_to_analyze:
    print(f"\nResults for {feature}:")
    result_df = pd.DataFrame({
        "value": results[feature]["value"],
        "MAE (W)": results[feature]["mae"],
        "MSE (W^2)": results[feature]["mse"],
        "RMSE (W)": results[feature]["rmse"],
        "MAPE (%)": results[feature]["mape"]
    })
    print(result_df)

    print("\nPercentile Errors:")
    percentiles_df = pd.DataFrame(results[feature]["percentiles"]).T
    percentiles_df.index.name = "Feature value"
    print(percentiles_df)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, regularizers
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

file_path = "original_preprocess.csv"
df = pd.read_csv(file_path)
df.drop(columns=["date", "failed_experiment", "cpu_platform", "UL/DL", "rapl_var", "n_pm", "pm_median", "txgain_ul", "clockspeed", "cqi_dl", "bsr_ul", "num_ues", "bler_dl"], inplace=True, errors='ignore')

features_to_analyze = ["txgain_ul", "selected_mcs_ul", "selected_mcs_dl", "airtime_ul", "airtime_dl"]
target_variable = "pm_power"

results = {
    feature: {"value": [], "mae": [], "mse": [], "rmse": [], "mape": [],
              "relative_errors": [], "mean_actual": [], "mean_predicted": [],
              "std_relative_errors": [], "percentiles": {}}
    for feature in features_to_analyze
}

#advanced regularised DNN model
def build_advanced_model(input_shape):
    model = tf.keras.Sequential([
        layers.Input(shape=(input_shape,)),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dense(1)
    ])
    return model

for feature in features_to_analyze:
    print(f"Processing feature: {feature}")
    unique_values = sorted(df[feature].unique())

    for value in unique_values:
        print(f"Processing value: {value}")
        df_value = df[df[feature] == value]

        if len(df_value) < 50:
            print(f"Skipping value {value} due to not enough data.")
            continue

        X = df_value.drop(columns=[target_variable] + features_to_analyze)
        y = df_value[target_variable]
        X = X.select_dtypes(include=[np.number])
        scaler = StandardScaler()
        X = scaler.fit_transform(X)

        #80-20 train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)
        model = build_advanced_model(X_train.shape[1])
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

        #early stopping callback
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=10, restore_best_weights=True
        )

        history = model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=100,
            batch_size=32,
            callbacks=[early_stopping],
            verbose=0
        )
        y_pred = model.predict(X_test).flatten()

        if np.isnan(y_pred).any():
            continue

        #calculate metrics
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
        relative_errors = np.abs((y_test - y_pred) / y_test) * 100
        mean_actual = np.mean(y_test)
        mean_predicted = np.mean(y_pred)
        std_relative_errors = np.std(relative_errors)

        #store results
        results[feature]["value"].append(value)
        results[feature]["mae"].append(mae)
        results[feature]["mse"].append(mse)
        results[feature]["rmse"].append(rmse)
        results[feature]["mape"].append(mape)
        results[feature]["relative_errors"].append(np.mean(relative_errors))
        results[feature]["std_relative_errors"].append(std_relative_errors)
        results[feature]["mean_actual"].append(mean_actual)
        results[feature]["mean_predicted"].append(mean_predicted)
        results[feature]["percentiles"][value] = {
            "10th Percentile": np.percentile(relative_errors, 10),
            "25th Percentile": np.percentile(relative_errors, 25),
            "50th Percentile (Median)": np.percentile(relative_errors, 50),
            "75th Percentile": np.percentile(relative_errors, 75),
            "90th Percentile": np.percentile(relative_errors, 90),
        }

for feature in features_to_analyze:
    plt.figure(figsize=(12, 6))

    values = results[feature]["value"]
    mean_relative_errors = results[feature]["relative_errors"]
    std_relative_errors = results[feature]["std_relative_errors"]

    plt.errorbar(values, mean_relative_errors, yerr=std_relative_errors, fmt='o-', capsize=5, label='Relative Error (%)', color='blue')
    plt.xlabel("Parameter groups")
    plt.ylabel("Mean Relative Error (%)")
    plt.legend()
    plt.grid()
    plt.show()

for feature in features_to_analyze:
    plt.figure(figsize=(12, 6))

    values = results[feature]["value"]
    mean_actual = results[feature]["mean_actual"]
    mean_predicted = results[feature]["mean_predicted"]
    std_predicted = results[feature]["std_relative_errors"]

    plt.errorbar(values, mean_actual, yerr=std_predicted, fmt='o-', capsize=5, label='Actual Power (W)', color='red')
    plt.errorbar(values, mean_predicted, yerr=std_predicted, fmt='s-', capsize=5, label='Predicted Power (W)', color='green')
    plt.xlabel("Parameter groups")
    plt.ylabel("Power (W)")
    plt.legend()
    plt.grid()
    plt.show()

for feature in features_to_analyze:
    plt.figure(figsize=(12, 6))

    values = results[feature]["value"]
    percentile_data = [results[feature]["percentiles"][value] for value in values]

    p10 = [p["10th Percentile"] for p in percentile_data]
    p25 = [p["25th Percentile"] for p in percentile_data]
    p50 = [p["50th Percentile (Median)"] for p in percentile_data]
    p75 = [p["75th Percentile"] for p in percentile_data]
    p90 = [p["90th Percentile"] for p in percentile_data]

    #percentiles
    plt.plot(values, p10, 'm--', label='10th Percentile')
    plt.plot(values, p25, 'b--', label='25th Percentile')
    plt.plot(values, p50, 'g-', label='50th Percentile (Median)')
    plt.plot(values, p75, 'r--', label='75th Percentile')
    plt.plot(values, p90, 'c--', label='90th Percentile')
    plt.xlabel("Parameter groups")
    plt.ylabel("Relative Error (%)")
    plt.legend()
    plt.grid()
    plt.show()

for feature in features_to_analyze:
    print(f"\nResults for {feature}:")
    result_df = pd.DataFrame({
        "value": results[feature]["value"],
        "MAE (W)": results[feature]["mae"],
        "MSE (W^2)": results[feature]["mse"],
        "RMSE (W)": results[feature]["rmse"],
        "MAPE (%)": results[feature]["mape"]
    })
    print(result_df)

    print("\nPercentile Errors:")
    percentiles_df = pd.DataFrame(results[feature]["percentiles"]).T
    percentiles_df.index.name = "Feature value"
    print(percentiles_df)






In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, regularizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import random
import json
import os
import pickle

SEED = 99
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)


file_path = "original_preprocess.csv"
df = pd.read_csv(file_path)


df.drop(columns=["date", "failed_experiment", "cpu_platform", "UL/DL", "rapl_var", "n_pm", "pm_median", "txgain_ul", "clockspeed", "cqi_dl", "bsr_ul", "num_ues", "bler_dl"], inplace=True, errors='ignore')


#feature engineering
df['txgain_airtime_dl'] = df['txgain_dl'] * df['selected_airtime_dl']
df['txgain_airtime_ul'] = df['txgain_ul'] * df['selected_airtime_ul']
df['mcs_dl_airtime'] = df['selected_mcs_dl'] * df['selected_airtime_dl']
df['mcs_ul_airtime'] = df['selected_mcs_ul'] * df['selected_airtime_ul']
df['mcs_gap'] = df['selected_mcs_dl'] - df['selected_mcs_ul']
df['snr_per_bler'] = df['mean_snr_ul'] / (df['bler_ul'] + 1e-6)
df['throughput_total'] = df['thr_dl'] + df['thr_ul']
df['airtime_total'] = df['airtime_dl'] + df['airtime_ul']

features_to_analyze = ["txgain_ul", "selected_mcs_ul", "selected_mcs_dl", "airtime_ul", "airtime_dl"]

target_variable = "pm_power"

resultsdx = {
    feature: {
        "value": [], "mae": [], "mse": [], "rmse": [], "mape": [],
        "relative_errors": [], "mean_actual": [], "mean_predicted": [],
        "std_relative_errors": [], "percentiles": {}
    }
    for feature in features_to_analyze
}


hyperparams = {
    "dense_units_1": 587,
    "l2_1": 0.000511,
    "dropout_1": 0.39,
    "dense_units_2": 261,
    "l2_2": 0.00563,
    "dropout_2": 0.402,
    "dense_units_3": 186,
    "dense_units_4": 99,
    "learning_rate": 0.00237,
    "epochs": 263,
    "batch_size": 32,
    "xgb_n_estimators": 256,
    "xgb_max_depth": 5,
    "xgb_learning_rate": 0.2276,
    "xgb_subsample": 0.5006,
    "xgb_colsample_bytree": 0.8246,
    "xgb_reg_lambda": 4.5554,
    "xgb_reg_alpha": 3.5152
}


#hybrid DNN-XGBoost model
def build_optimized_model(input_shape):
    model = tf.keras.Sequential([
        layers.Input(shape=(input_shape,)),
        layers.Dense(hyperparams["dense_units_1"], activation='relu',
                     kernel_regularizer=regularizers.l2(hyperparams["l2_1"])),
        layers.BatchNormalization(),
        layers.Dropout(hyperparams["dropout_1"]),

        layers.Dense(hyperparams["dense_units_2"], activation='relu',
                     kernel_regularizer=regularizers.l2(hyperparams["l2_2"])),
        layers.BatchNormalization(),
        layers.Dropout(hyperparams["dropout_2"]),

        layers.Dense(hyperparams["dense_units_3"], activation='relu'),
        layers.Dense(hyperparams["dense_units_4"], activation='relu')
    ])
    return model


#compile optimised DNN model
def compile_model(model):
    optimizer = tf.keras.optimizers.Adam(learning_rate=hyperparams["learning_rate"])
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model


#loop through each feature
for feature in features_to_analyze:
    print(f"Processing feature: {feature}")
    unique_values = sorted(df[feature].unique())
    for value in unique_values:
        print(f"Processing value: {value}")
        df_value = df[df[feature] == value]
        if len(df_value) < 50:
            print(f"Skipping value {value} due to not enough data.")
            continue

        X = df_value.drop(columns=[target_variable] + features_to_analyze)
        y = df_value[target_variable]
        X = X.select_dtypes(include=[np.number])
        scaler = StandardScaler()
        X = scaler.fit_transform(X)

        #80-20 train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=99 )
        dnn_model = build_optimized_model(X_train.shape[1])
        dnn_model = compile_model(dnn_model)

        #train the DNN model
        dnn_model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=hyperparams["epochs"],
            batch_size=hyperparams["batch_size"],
            verbose=0
        )
        X_train_features = dnn_model.predict(X_train)
        X_test_features = dnn_model.predict(X_test)

        #train XGBoost on the extracted features
        xgb_model = XGBRegressor(
            objective='reg:squarederror',
            n_estimators=hyperparams["xgb_n_estimators"],
            learning_rate=hyperparams["xgb_learning_rate"],
            max_depth=hyperparams["xgb_max_depth"],
            subsample=hyperparams["xgb_subsample"],
            colsample_bytree=hyperparams["xgb_colsample_bytree"],
            reg_lambda=hyperparams["xgb_reg_lambda"],
            reg_alpha=hyperparams["xgb_reg_alpha"],
            random_state=99
        )
        xgb_model.fit(X_train_features, y_train)
        y_pred = xgb_model.predict(X_test_features)

        if np.isnan(y_pred).any():
            continue

        #calculate metrics
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
        relative_errors = np.abs((y_test - y_pred) / y_test) * 100

        #results
        resultsdx[feature]["value"].append(value)
        resultsdx[feature]["mae"].append(mae)
        resultsdx[feature]["mse"].append(mse)
        resultsdx[feature]["rmse"].append(rmse)
        resultsdx[feature]["mape"].append(mape)
        resultsdx[feature]["relative_errors"].append(np.mean(relative_errors))
        resultsdx[feature]["mean_actual"].append(np.mean(y_test))
        resultsdx[feature]["mean_predicted"].append(np.mean(y_pred))
        resultsdx[feature]["std_relative_errors"].append(np.std(relative_errors))
        resultsdx[feature]["percentiles"][value] = {
            "10th Percentile": np.percentile(relative_errors, 10),
            "25th Percentile": np.percentile(relative_errors, 25),
            "50th Percentile (Median)": np.percentile(relative_errors, 50),
            "75th Percentile": np.percentile(relative_errors, 75),
            "90th Percentile": np.percentile(relative_errors, 90),
        }

for feature in features_to_analyze:
    print(f"\nResults for {feature}:")
    result_df = pd.DataFrame({
        "value": resultsdx[feature]["value"],
        "MAE (W)": resultsdx[feature]["mae"],
        "MSE (W^2)": resultsdx[feature]["mse"],
        "RMSE (W)": resultsdx[feature]["rmse"],
        "MAPE (%)": resultsdx[feature]["mape"]
    })
    print(result_df)

    print("\nPercentile Errors:")
    percentiles_df = pd.DataFrame(resultsdx[feature]["percentiles"]).T
    percentiles_df.index.name = "Feature value"
    print(percentiles_df)

for feature in features_to_analyze:
    plt.figure(figsize=(12, 6))
    plt.errorbar(resultsdx[feature]["value"], resultsdx[feature]["relative_errors"],
                 yerr=resultsdx[feature]["std_relative_errors"], fmt='o-', capsize=5)
    plt.xlabel(feature)
    plt.ylabel("Mean Relative Error (%)")
    plt.grid()
    plt.show()

for feature in features_to_analyze:
    plt.figure(figsize=(12, 6))
    plt.errorbar(resultsdx[feature]["value"], resultsdx[feature]["mean_actual"],
                 yerr=resultsdx[feature]["std_relative_errors"], fmt='o-', capsize=5, label='Actual Power', color='red')
    plt.errorbar(resultsdx[feature]["value"], resultsdx[feature]["mean_predicted"],
                 yerr=resultsdx[feature]["std_relative_errors"], fmt='s-', capsize=5, label='Predicted Power', color='green')
    plt.xlabel(feature)
    plt.ylabel("Power (W)")
    plt.legend()
    plt.grid()
    plt.show()
    plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt


df = pd.read_csv("original_preprocess.csv")


df["tx_ul"] = df["txgain_ul"] * df["airtime_ul"]
df["tx_dl"] = df["txgain_dl"] * df["airtime_dl"]

#analytical model - Linear gain-airtime
def model_gain(tx_ul, tx_dl):
    alpha, beta, gamma = 0.03, 0.01, 12
    return alpha * tx_ul + beta * tx_dl + gamma

#analytical model 2 - Airtime-based quadratic
def model_airtime(airtime_ul, airtime_dl):
    P_idle = 6
    b1_ul, b2_ul = 4, 5
    b1_dl, b2_dl = 3, 4
    return (P_idle +
            b1_ul * airtime_ul + b2_ul * airtime_ul**2 +
            b1_dl * airtime_dl + b2_dl * airtime_dl**2)


df["pred_power_gain"] = model_gain(df["tx_ul"], df["tx_dl"])
df["pred_power_airtime"] = model_airtime(df["airtime_ul"], df["airtime_dl"])

group_features = ["txgain_ul", "selected_mcs_ul", "selected_mcs_dl", "airtime_ul", "airtime_dl"]

model_info = {
    "Gain Model": "pred_power_gain",
    "Airtime Model": "pred_power_airtime"
}

for model_name, pred_col in model_info.items():
    print(f"\ Evaluation for: {model_name}")

    for feature in group_features:
        print(f"\n Grouped by '{feature}'")
        summary = []

        for val, group in df.groupby(feature):
            if len(group) < 50:
                continue

            y_true = group["pm_power"]
            y_pred = group[pred_col]
            mre = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
            summary.append({
                feature: val,
                "Mean Relative Error (%)": mre
            })
        plot_df = pd.DataFrame(summary)
        print(plot_df.to_string(index=False))

        plt.figure(figsize=(10, 4))
        plt.title(f"{model_name} - MRE (%) grouped by {feature}")
        plt.xlabel(feature)
        plt.ylabel("Mean Relative Error (%)")
        plt.grid(True)
        plt.tight_layout()
        plt.show()



In [ ]:
file_path = 'original_preprocess.csv'
df = pd.read_csv(file_path)

#config params
config_features = [
    'txgain_ul',
    'selected_mcs_dl',
    'selected_mcs_ul',
    'selected_airtime_dl',
    'selected_airtime_ul',
]

group_counts = df.groupby(config_features).size().reset_index(name='count')
repeated_configs = group_counts[group_counts['count'] >= 2]
df_repeats = df.merge(repeated_configs[config_features], on=config_features, how='inner')
std_devs = df_repeats.groupby(config_features)['pm_power'].std().dropna()
intrinsic_noise = std_devs.mean()
mean_pm_power = df['pm_power'].mean()


intrinsic_noise_percent = (intrinsic_noise / mean_pm_power) * 100
print(f"Estimated intrinsic noise (std dev of power consumption): {intrinsic_noise:.4f} W")
print(f"Intrinsic noise as percentage of mean power consumption: {intrinsic_noise_percent:.2f}%")
